## Attention LSTM 
    - 30 day Rolling Window Forecast using Attention LSTM

In [1]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
# from attention_decoder import AttentionDecoder

Using TensorFlow backend.


In [18]:
import keras
print(keras.__version__)

2.3.1


In [3]:
import pandas as pd
from sklearn .preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM

In [4]:
from math import sqrt
from numpy import split
from numpy import array
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [5]:
from matplotlib import pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
pd.options.display.float_format = '{:,.10f}'.format

In [6]:
data  = pd.read_excel('emotion.xls').drop('Date', axis=1 ).set_index("day")

In [7]:
data = data.astype('float32')

In [8]:
data_vol =  data.iloc[ : , 0:13].drop(['Close_nifty', 'Close_sensex','% change_nifty', '% change_sensex'], axis= 1)

**DATA PREP**

In [9]:
scaler = MinMaxScaler()
train, test = data_vol[18:3138], data_vol[3138:3168]
train = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)

In [10]:
yscaler = MinMaxScaler()
df = pd.DataFrame(data_vol.iloc[18:3138, 8])
df = pd.DataFrame(yscaler.fit_transform(df), columns=df.columns, index=df.index)

In [11]:
train = array(split(train.values, len(train)/30))
test = array(split(test.values, len(test)/30))

In [12]:
def to_supervised(train, n_input, n_out=30):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	for _ in range(len(data)):
		in_end = in_start + n_input
		out_end = in_end + n_out
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 8])
		in_start += 1
	return array(X), array(y)

**MODEL**

In [28]:
from keras.layers.core import Layer  
from keras import initializers, regularizers, constraints  
from keras import backend as K

In [32]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
        
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape = (input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape = (input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [45]:
def build_model(train, n_input):
	train_x, train_y = to_supervised(train, n_input)
	verbose, epochs, batch_size = 1, 55, 20
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
	print(train_x.shape)
	print(train_y.shape)
    
	units= 200
	model = Sequential() 
	model.add(LSTM(units, activation='relu', input_shape=(n_timesteps, n_features), return_sequences = True))#output:(none, timesteps,units)
	model.add(Attention())
	model.add(RepeatVector(n_outputs))
	model.add(Dense(1))
	model.compile(loss='mse', optimizer='adam')
	model.summary()

    
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

In [47]:
n_input = 30 
model = build_model(train, n_input) 
model.save( 'model8' + '.h5')
print("model saved")

(3061, 30, 9)
(3061, 30, 1)
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 30, 200)           168000    
_________________________________________________________________
attention_9 (Attention)      (None, 200)               230       
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 30, 200)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 30, 1)             201       
Total params: 168,431
Trainable params: 168,431
Non-trainable params: 0
_________________________________________________________________
Epoch 1/55
3061/3061 [==============================] - 24s 8ms/step - loss: 0.0074
Epoch 2/55
3061/3061 [==============================] - 24s 8ms/step - loss: 0.0060
Epoch 3/55
3061/3061 [===========================

## Forecast

In [49]:
from numpy import loadtxt
from keras.models import load_model  
from keras.utils.vis_utils import plot_model

In [50]:
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=1)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

In [51]:
def evaluate_forecasts_new(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
        
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

In [52]:
def evaluate_model_new(train, test, n_input):
    
	# history is a list of weekly data
	history = [x for x in train]
    
	# walk-forward validation 
	predictions = list()
    
	for i in range(len(test)):
		yhat_sequence = forecast(model, history, n_input)   
		predictions.append(yhat_sequence)
		history.append(test[i, :])
        
	# evaluate predictions days for each week
	predictions = array(predictions)
	return predictions

In [60]:
preds = evaluate_model_new(train, test, 30)

1/1 [==============================] - 0s 40ms/step


In [54]:
preds.shape
predictions = yscaler.inverse_transform(preds[0,:, :])

In [56]:
score, scores = evaluate_forecasts_new(test[:, :, 8], predictions)

In [57]:
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

In [58]:
summarize_scores('test', score, scores)

test: [2.607] 0.1, 0.4, 0.4, 0.7, 0.6, 0.6, 0.5, 0.4, 0.4, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.1, 0.4, 0.4, 0.4, 0.3, 1.4, 1.3, 1.2, 1.1, 1.0, 1.3, 3.5, 3.2, 9.0, 9.4


### **References** :

To implement Attention Mechanism I used the [source code](https://gist.github.com/cbaziotis/6428df359af27d58078ca5ed9792bd6d) of Christos Baziotis. A few changes very made to debug errors [using this gist comment](https://gist.github.com/cbaziotis/6428df359af27d58078ca5ed9792bd6d#gistcomment-3094589 )